In [97]:
import yaml
import glob
from shortid import ShortId
from rdflib import plugin, ConjunctiveGraph, Graph, URIRef, Literal, BNode
from rdflib.store import Store
from rdflib.namespace import RDF, RDFS, XSD

ns = 'http://envri.eu/ns/'
sid = ShortId()
store = plugin.get('IOMemory', Store)()

vocab = dict()
vocab['Bag'] = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#Bag')
vocab['li'] = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#li')
vocab['ResearchInfrastructure'] = URIRef('http://www.oil-e.net/ontology/envri-rm.owl#ResearchInfrastructure')
vocab['Repository'] = URIRef('http://envri.eu/ns/Repository')
vocab['data_repository'] = URIRef('http://envri.eu/ns/DataRepository')
vocab['metadata_repository'] = URIRef('http://envri.eu/ns/MetadataRepository')
vocab['site_repository'] = URIRef('http://envri.eu/ns/SiteRepository')

vocab['Identifier'] = URIRef('http://envri.eu/ns/Identifier')
vocab['LocalIdentifier'] = URIRef('http://envri.eu/ns/LocalIdentifier')
vocab['local_ID'] = URIRef('http://envri.eu/ns/LocalIdentifier')
vocab['PersistentIdentifier'] = URIRef('http://envri.eu/ns/PersistentIdentifier')
vocab['PID'] = URIRef('http://envri.eu/ns/PersistentIdentifier')
vocab['DigitalObjectIdentifier'] = URIRef('http://envri.eu/ns/DigitalObjectIdentifier')
vocab['DOI'] = URIRef('http://envri.eu/ns/DigitalObjectIdentifier')
vocab['AccessMechanism'] = URIRef('http://envri.eu/ns/AccessMechanism')

vocab['date'] = URIRef('http://purl.org/dc/terms/date')
vocab['version'] = URIRef('http://purl.org/dc/terms/hasVersion')
vocab['creator'] = URIRef('http://purl.org/dc/terms/creator')
vocab['name'] = URIRef('http://xmlns.com/foaf/0.1/name')
vocab['email'] = URIRef('http://xmlns.com/foaf/0.1/mbox')
vocab['label'] = URIRef('http://www.w3.org/2000/01/rdf-schema#label')

vocab['acronym'] = URIRef('http://envri.eu/ns/acronym')
vocab['url'] = URIRef('http://envri.eu/ns/url')
vocab['hasDataManagementPlans'] = URIRef('http://envri.eu/ns/hasDataManagementPlans')
vocab['usesSpecificDataManagementPlanTools'] = URIRef('http://envri.eu/ns/usesSpecificDataManagementPlanTools')
vocab['appliedDataPublishingSteps'] = URIRef('http://envri.eu/ns/appliedDataPublishingSteps')
vocab['hasRepository'] = URIRef('http://envri.eu/ns/hasRepository')
vocab['usesSystem'] = URIRef('http://envri.eu/ns/usesSystem')
vocab['usesIdentifier'] = URIRef('http://envri.eu/ns/usesIdentifier')
vocab['usesProvider'] = URIRef('http://envri.eu/ns/usesProvider')
vocab['isAssigned'] = URIRef('http://envri.eu/ns/isAssigned')
vocab['hasCertificationMethod'] = URIRef('http://envri.eu/ns/hasCertificationMethod')
vocab['hasPolicy'] = URIRef('http://envri.eu/ns/hasPolicy')
vocab['inRegistry'] = URIRef('http://envri.eu/ns/inRegistry')
vocab['hasPersistentPolicy'] = URIRef('http://envri.eu/ns/hasPersistentPolicy')
vocab['hasPersistencyGuaranty'] = URIRef('http://envri.eu/ns/hasPersistencyGuaranty')
vocab['hasAccessMechanisms'] = URIRef('http://envri.eu/ns/hasAccessMechanisms')
vocab['hasAuthenticationMethod'] = URIRef('http://envri.eu/ns/hasAuthenticationMethod')
vocab['hasAccessProtocolUrl'] = URIRef('http://envri.eu/ns/hasAccessProtocolUrl')
vocab['protocolIsOpen'] = URIRef('http://envri.eu/ns/protocolIsOpen')
vocab['protocolIsRoyaltyFree'] = URIRef('http://envri.eu/ns/protocolIsRoyaltyFree')
vocab['maintainsOwnUserDatabase'] = URIRef('http://envri.eu/ns/maintainsOwnUserDatabase')
vocab['usesORCIDinAAI'] = URIRef('http://envri.eu/ns/usesORCIDinAAI')
vocab['supportsMajorAccessTechnology'] = URIRef('http://envri.eu/ns/supportsMajorAccessTechnology')
vocab['usesAuthorisationTechnique'] = URIRef('http://envri.eu/ns/usesAuthorisationTechnique')
vocab['usesAuthorisationFor'] = URIRef('http://envri.eu/ns/usesAuthorisationFor')
vocab['authorisationRequired'] = URIRef('http://envri.eu/ns/authorisationRequired')
vocab['hasAccessConcentProcessDescriptionUri'] = URIRef('http://envri.eu/ns/hasAccessConcentProcessDescriptionUri')
vocab['usesDataLicenses'] = URIRef('http://envri.eu/ns/usesDataLicenses')
vocab['dataLicenseIri'] = URIRef('http://envri.eu/ns/dataLicenseIri')

vocab['local'] = URIRef('http://envri.eu/ns/Local')
vocab['local_license'] = URIRef('http://envri.eu/ns/LocalLicense')
vocab['download_data'] = URIRef('http://envri.eu/ns/DataDownload')
vocab['upload_data'] = URIRef('http://envri.eu/ns/DataUpload')
vocab['Marine.id'] = URIRef('http://envri.eu/ns/Marine.id')
vocab['OAUTH'] = URIRef('http://envri.eu/ns/OAUTH')
vocab['CAS'] = URIRef('http://envri.eu/ns/CAS')
vocab['Zenodo'] = URIRef('http://envri.eu/ns/Zenodo')
vocab['planned'] = URIRef('http://envri.eu/ns/Planned')
vocab['Sextant-Si'] = URIRef('http://envri.eu/ns/Sextant-Si')
vocab['52_NORTH_SOS'] = URIRef('http://envri.eu/ns/52_NORTH_SOS')
vocab['Handle'] = URIRef('http://envri.eu/ns/Handle')
vocab['local_system'] = URIRef('http://envri.eu/ns/LocalSystem')
vocab['local_service'] = URIRef('http://envri.eu/ns/LocalService')
vocab['local_LDAP'] = URIRef('http://envri.eu/ns/LocalLDAP')
vocab['DEIMS-SDR'] = URIRef('http://envri.eu/ns/DEIMS-SDR')
vocab['DataCite'] = URIRef('http://envri.eu/ns/DataCite')
vocab['re3data'] = URIRef('http://envri.eu/ns/re3data')
vocab['none'] = URIRef('http://envri.eu/ns/None')
vocab[None] = URIRef('http://envri.eu/ns/NULL')

def _l(g, d, n, k, t):
    if isinstance(k, list):
        p = vocab[k[0]]
        o = d[k[1]]
    else:
        p = vocab[k]
        o = d[k]

    if o == 'VOID':
        return
    
    g.add((n, p, Literal(o, datatype=t)))
    
def _b(g, n, k, b):
    g.add((n, vocab[k], b))
    
def _r(g, d, n, k):
    if isinstance(k, list):
        p = vocab[k[0]]
        o = d[k[1]]
    else:
        p = vocab[k]
        o = d[k]
        
    if o == 'VOID':
        return
    
    if o is None:
        g.add((n, p, URIRef(vocab[o])))
        return
    
    if o.find('http') > -1 or o.find('www') > -1 or o.find('@') > -1:
        g.add((n, p, URIRef(o)))
        return
        
    g.add((n, p, URIRef(vocab[o])))
    
def _t(g, n, k):
    g.add((n, RDF.type, vocab[k]))
    
def _li(g, n, e):
    g.add((n, vocab['li'], URIRef(vocab[e])))
    
def process(d):
    gid = URIRef('{}G{}'.format(ns, sid.generate()))
    g = Graph(store, gid)
    process_survey(g, d['survey'])
    process_infrastructure(g, d['infrastructure'])
    
def process_survey(g, d):
    n = BNode()
    _l(g, d, g.identifier, 'date', XSD.date)
    _l(g, d, g.identifier, 'version', XSD.string)
    process_creator(g, d['creator'], g.identifier)
    
def process_creator(g, d, n):
    n1 = BNode()
    _b(g, n, 'creator', n1)
    _l(g, d, n1, 'name', XSD.string)
    _r(g, d, n1, 'email')
    
def process_infrastructure(g, d):
    n = BNode()
    _t(g, n, 'ResearchInfrastructure')
    _l(g, d, n, 'acronym', XSD.string)
    _l(g, d, n, ['label', 'name'], XSD.string)
    _r(g, d, n, ['url', 'recognized authority IRI'])
    process_infrastructure_datamanagementplans(g, d['data management plans'], n)
    for repository in d['repositories']:
        process_repository(g, repository, n)
    
def process_infrastructure_datamanagementplans(g, d, n):
    n1 = BNode()
    _b(g, n, 'hasDataManagementPlans', n1)
    _l(g, d, n1, ['usesSpecificDataManagementPlanTools', 'specific DMP tools used'], XSD.bool)
    _l(g, d, n1, ['appliedDataPublishingSteps', 'data publishing steps applied'], XSD.string)
        
def process_repository(g, d, n):
    n1 = BNode()
    _b(g, n, 'hasRepository', n1)
    _t(g, n1, 'Repository')
    _l(g, d, n1, ['label', 'name'], XSD.string)  
    _r(g, d, n1, ['url', 'IRI'])
    _t(g, n1, d['kind'])
    _r(g, d, n1, ['usesSystem', 'software'])
    process_repository_identifier(g, d['identifier'], n1)
    _l(g, d, n1, ['hasCertificationMethod', 'certification method'], XSD.string)
    _l(g, d, n1, ['hasPolicy', 'policy'], XSD.string)
    _r(g, d, n1, ['inRegistry', 'registry'])
    _l(g, d, n1, ['hasPersistencyGuaranty', 'persistency-guaranty'], XSD.string)
    process_repository_access(g, d['access mechanisms'], n1)
    process_repository_data(g, d['data'], n1)
    process_repository_metadata(g, d['metadata'], n1)
    process_repository_vocabularies(g, d['vocabularies'], n1)
    process_repository_dataprocessing(g, d['data processing'], n1)
    process_repository_fairness(g, d['fairness'], n1)
    process_repository_testfairness(g, d['test fairness'], n1)
        
        
def process_repository_identifier(g, d, n):
    n1 = BNode()
    _b(g, n, 'usesIdentifier', n1)
    _t(g, n1, 'Identifier')
    _t(g, n1, d['kind'])
    _r(g, d, n1, ['usesSystem', 'system'])
    _l(g, d, n1, ['isAssigned', 'assigned'], XSD.string)
    _r(g, d, n1, ['usesProvider', 'provider'])
    
    
def process_repository_access(g, d, n):
    n1 = BNode()
    _b(g, n, 'hasAccessMechanisms', n1)
    _t(g, n1, 'AccessMechanism')
    _l(g, d, n1, ['hasAuthenticationMethod', 'authentication method'], XSD.string)
    _r(g, d, n1, ['hasAccessProtocolUrl', 'access protocol URI'])
    _l(g, d, n1, ['protocolIsOpen', 'protocol open'], XSD.bool)
    _l(g, d, n1, ['protocolIsRoyaltyFree', 'protocol royalty free'], XSD.bool)
    _l(g, d, n1, ['maintainsOwnUserDatabase', 'own user database maintained'], XSD.bool)
    _l(g, d, n1, ['usesORCIDinAAI', 'ORCID used in AAI'], XSD.bool)
    _r(g, d, n1, ['supportsMajorAccessTechnology', 'major access technology supported'])
    _r(g, d, n1, ['usesAuthorisationTechnique', 'authorisation technique'])
    process_authorisation_needed_for(g, d['authorisation needed for'], n1)
    _l(g, d, n1, ['authorisationRequired', 'authorization for accessing content needed'], XSD.bool)
    _r(g, d, n1, ['hasAccessConcentProcessDescriptionUri', 'access content process description IRI'])
    _r(g, d, n1, ['usesDataLicenses', 'data licenses in use'])
    _r(g, d, n1, ['dataLicenseIri', 'data license IRI'])
    
    
def process_authorisation_needed_for(g, d, n):
    if d is None:
        return
    n1 = BNode()
    _b(g, n, 'usesAuthorisationFor', n1)
    _t(g, n1, 'Bag')
    for e in d:
        _li(g, n1, e)
    
def process_repository_data(g, d, n):
    return
    

def process_repository_metadata(g, d, n):
    return


def process_repository_vocabularies(g, d, n):
    return


def process_repository_dataprocessing(g, d, n):
    return


def process_repository_fairness(g, d, n):
    return


def process_repository_testfairness(g, d, n):
    return

    
for file in glob.glob('descriptions/*.yaml'):
    with open(file, 'r') as f:
        for document in yaml.load_all(f, Loader=yaml.FullLoader):
            process(document)
    
g = ConjunctiveGraph(store)
g.bind('envri', ns)
g.bind('dcterms', 'http://purl.org/dc/terms/')
g.bind('foaf', 'http://xmlns.com/foaf/0.1/')
g.serialize(destination='data.trig', format='trig')

KeyError: 'data licenses in use'

In [63]:
import io
import pandas as pd
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = io.BytesIO()
    serializer.serialize(output)
    display(pd.read_csv(io.StringIO(output.getvalue().decode('utf-8'))))

In [66]:
query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?ri_acronym ?ri_url ?rep_label ?rep_soft WHERE {
    ?g dcterms:date ?date .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri envri:acronym ?ri_acronym . 
        ?ri envri:url ?ri_url .
        ?ri envri:hasRepository ?rep .
        ?rep a envri:Repository .
        ?rep rdfs:label ?rep_label .
        OPTIONAL { ?rep envri:usesSystem ?rep_sys } .
    }
    FILTER (?date > "2019-03-15"^^xsd:date)
}
""")

,date,ri_acronym,ri_url,rep_label,rep_soft
0,2019-03-29,SDN,www.seadatanet.org,Data Products Catalogue,http://envri.eu/ns/Sextant-Si?
1,2019-03-29,SDN,www.seadatanet.org,SDN metadata,NaN
2,2019-03-19,eLTER,http://www.lter-europe.net/elter,EUDAT/FZJ B2SHARE,http://envri.eu/ns/Zenodo
3,2019-03-19,eLTER,http://www.lter-europe.net/elter,eLTER CDN,http://envri.eu/ns/52NORTHSOS
4,2019-03-19,eLTER,http://www.lter-europe.net/elter,DEIMS-SDR,NaN


In [64]:
query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?ri_acronym ?ri_url ?rep_label ?rep_soft WHERE {
    ?g dcterms:date ?date .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri envri:acronym ?ri_acronym . 
        ?ri envri:url ?ri_url .
        ?ri envri:hasRepository ?rep .
        ?rep a envri:MetadataRepository .
        ?rep rdfs:label ?rep_label .
        OPTIONAL { ?rep envri:usesSystem ?rep_sys } .
    }
    FILTER (?date > "2019-03-15"^^xsd:date)
}
""")

,date,ri_acronym,ri_url,rep_label,rep_soft
0,2019-03-29,SDN,www.seadatanet.org,SDN metadata,NaN
1,2019-03-19,eLTER,http://www.lter-europe.net/elter,DEIMS-SDR,NaN


In [68]:
query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?ri_acronym ?ri_url ?rep_label ?rep_soft WHERE {
    ?g dcterms:date ?date .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri envri:acronym ?ri_acronym . 
        ?ri envri:url ?ri_url .
        ?ri envri:hasRepository ?rep .
        ?rep a envri:DataRepository .
        ?rep rdfs:label ?rep_label .
        OPTIONAL { ?rep envri:usesSystem ?rep_sys } .
    }
    FILTER (?date > "2019-03-15"^^xsd:date)
}
""")

,date,ri_acronym,ri_url,rep_label,rep_soft
0,2019-03-29,SDN,www.seadatanet.org,Data Products Catalogue,http://envri.eu/ns/Sextant-Si?
1,2019-03-19,eLTER,http://www.lter-europe.net/elter,eLTER CDN,http://envri.eu/ns/52NORTHSOS
2,2019-03-19,eLTER,http://www.lter-europe.net/elter,EUDAT/FZJ B2SHARE,http://envri.eu/ns/Zenodo


In [69]:
query("""
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX envri: <http://envri.eu/ns/>
PREFIX rm: <http://www.oil-e.net/ontology/envri-rm.owl#>

SELECT ?date ?ri_acronym ?ri_url ?rep_label ?rep_sys WHERE {
    ?g dcterms:date ?date .
    GRAPH ?g { 
        ?ri a rm:ResearchInfrastructure .
        ?ri envri:acronym ?ri_acronym . 
        ?ri envri:url ?ri_url .
        ?ri envri:hasRepository ?rep .
        ?rep a envri:DataRepository .
        ?rep rdfs:label ?rep_label .
        OPTIONAL { ?rep envri:usesSystem ?rep_sys } .
    }
    FILTER (?date > "2019-03-25"^^xsd:date)
}
""")

,date,ri_acronym,ri_url,rep_label,rep_soft
0,2019-03-29,SDN,www.seadatanet.org,Data Products Catalogue,http://envri.eu/ns/Sextant-Si?
